In [1]:
import functools
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pprint
import tensorflow_model_analysis as tfma
from google.protobuf import text_format

In [2]:
print(tf.__version__)

2.4.1


In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [5]:
train_df = pd.read_csv(train_file_path, header='infer')
test_df = pd.read_csv(test_file_path, header='infer')
train_df.dropna()
test_df.dropna()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,35.0,0,0,8.0500,Third,unknown,Southampton,y
1,0,male,54.0,0,0,51.8625,First,E,Southampton,y
2,1,female,58.0,0,0,26.5500,First,C,Southampton,y
3,1,female,55.0,0,0,16.0000,Second,unknown,Southampton,y
4,1,male,34.0,0,0,13.0000,Second,D,Southampton,y
...,...,...,...,...,...,...,...,...,...,...
259,1,female,25.0,0,1,26.0000,Second,unknown,Southampton,n
260,0,male,33.0,0,0,7.8958,Third,unknown,Southampton,y
261,0,female,39.0,0,5,29.1250,Third,unknown,Queenstown,n
262,0,male,27.0,0,0,13.0000,Second,unknown,Southampton,y


In [6]:
train_df

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,0,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y
625,0,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [7]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

train_ds = tf.data.experimental.make_csv_dataset(
      train_file_path,
      batch_size=3,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)

In [8]:
test_ds = tf.data.experimental.make_csv_dataset(
      test_file_path,
      batch_size=3,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)

In [9]:
for batch, label in train_ds.take(1):
  print(label)
  for key, value in batch.items():
    print("{}: {}".format(key,value.numpy()))

tf.Tensor([1 1 0], shape=(3,), dtype=int32)
sex: [b'female' b'male' b'male']
age: [31. 45. 29.]
n_siblings_spouses: [1 0 1]
parch: [1 0 0]
fare: [26.25   26.55   27.7208]
class: [b'Second' b'First' b'Second']
deck: [b'unknown' b'unknown' b'unknown']
embark_town: [b'Southampton' b'Southampton' b'Cherbourg']
alone: [b'n' b'y' b'n']


In [10]:
feature_columns = []

# numeric cols
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
  feature_columns.append(feature_column.numeric_column(header))

In [11]:
train_df.describe()

,survived,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.387560,29.631308,0.545455,0.379585,34.385399
std,0.487582,12.511818,1.151090,0.792999,54.597730
min,0.000000,0.750000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,0.000000,7.895800
50%,0.000000,28.000000,0.000000,0.000000,15.045800
75%,1.000000,35.000000,1.000000,0.000000,31.387500
max,1.000000,80.000000,8.000000,5.000000,512.329200


In [12]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [13]:
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[23, 28, 35])
demo(age_buckets)

[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [14]:
h = {}
for col in train_df:
  if col in ['sex', 'class', 'deck', 'embark_town', 'alone']:
    print(col, ':', train_df[col].unique())
    h[col] = train_df[col].unique()

sex : ['male' 'female']
class : ['Third' 'First' 'Second']
deck : ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town : ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone : ['n' 'y']


In [15]:
type(h.get('sex').tolist())

list

In [16]:
sex_type = feature_column.categorical_column_with_vocabulary_list(
      'sex', ['male' 'female'])
sex_type_one_hot = feature_column.indicator_column(sex_type)

## use key to lookup value and pass the value to one-hot encoding.
sex_type = feature_column.categorical_column_with_vocabulary_list(
      'sex', h.get('sex').tolist())
sex_type_one_hot = feature_column.indicator_column(sex_type)

class_type = feature_column.categorical_column_with_vocabulary_list(
      'class', ['Third' 'First' 'Second'])
class_type_one_hot = feature_column.indicator_column(class_type)

deck_type = feature_column.categorical_column_with_vocabulary_list(
      'deck', h.get('deck').tolist())
deck_type_one_hot = feature_column.indicator_column(deck_type)

embark_town_type = feature_column.categorical_column_with_vocabulary_list(
      'embark_town', h.get('embark_town').tolist())
embark_town_type_one_hot = feature_column.indicator_column(embark_town_type)

alone_type = feature_column.categorical_column_with_vocabulary_list(
      'alone', h.get('alone').tolist())
alone_one_hot = feature_column.indicator_column(alone_type)

In [17]:
deck = feature_column.categorical_column_with_vocabulary_list(
      'deck', train_df.deck.unique())
deck_embedding = feature_column.embedding_column(deck, dimension=3)

In [18]:
class_hashed = feature_column.categorical_column_with_hash_bucket(
      'class', hash_bucket_size=4)
demo(feature_column.indicator_column(class_hashed))

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [19]:
feature_column

<module 'tensorflow._api.v2.feature_column' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/_api/v2/feature_column/__init__.py'>

In [20]:
crossed_feature = feature_column.crossed_column([sex_type, class_type], hash_bucket_size=5)
#demo(feature_column.indicator_column(crossed_feature))

In [21]:
crossed_feature

CrossedColumn(keys=(VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('ThirdFirstSecond',), dtype=tf.string, default_value=-1, num_oov_buckets=0)), hash_bucket_size=5, hash_key=None)

In [22]:
feature_columns = []

# numeric cols
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
  feature_columns.append(feature_column.numeric_column(header))

In [23]:
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[23, 28, 35])

In [24]:
feature_columns.append(age_buckets)

In [25]:
# indicator_columns
indicator_column_names = ['sex', 'class', 'deck', 'embark_town', 'alone']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, train_df[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [26]:
# append embedding columns
deck = feature_column.categorical_column_with_vocabulary_list(
      'deck', train_df.deck.unique())
deck_embedding = feature_column.embedding_column(deck, dimension=3)
feature_columns.append(deck_embedding)

In [27]:
# append crossed columns
cross_type_feature = feature_column.crossed_column(['sex', 'class'], hash_bucket_size=5)
feature_columns.append(feature_column.indicator_column(cross_type_feature))

In [28]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [29]:
def pandas_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('survived')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [30]:
val_df, test_df = train_test_split(test_df, test_size=0.4)

In [31]:
print(len(train_df), 'train examples')
print(len(val_df), 'validation examples')
print(len(test_df), 'test examples')

627 train examples
158 validation examples
106 test examples


In [32]:
train_df

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,0,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y
625,0,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [33]:
'''
batch_size = 33
train_ds = df_to_dataset(train_df, batch_size=batch_size)
'''

'\nbatch_size = 33\ntrain_ds = df_to_dataset(train_df, batch_size=batch_size)\n'

In [34]:
batch_size = 33
labels = train_df.copy().pop('survived')
working_ds = tf.data.Dataset.from_tensor_slices((dict(train_df), labels))
working_ds = working_ds.shuffle(buffer_size=len(train_df))
train_ds = working_ds.batch(batch_size)

In [35]:
val_ds = pandas_to_dataset(val_df, shuffle=False, batch_size=batch_size)
test_ds = pandas_to_dataset(test_df, shuffle=False, batch_size=batch_size)

In [36]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


 1/19 [>.............................] - ETA: 24s - loss: 2.6612 - accuracy: 0.5758WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'sex': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=string>, 'age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'n_siblings_spouses': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=int64>, 'parch': <tf.Tensor 'ExpandDims_7:0' shape=(None, 1) dtype=int64>, 'fare': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'class': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=string>, 'deck': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'embark_town': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=string>, 'alone': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=string>}
Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


19/19 [==============================] - 2s 45ms/step - loss: 1.6756 - accuracy: 0.5362 - val_loss: 0.6373 - val_accuracy: 0.6392
Epoch 2/10
19/19 [==============================] - 0s 4ms/step - loss: 0.9282 - accuracy: 0.5896 - val_loss: 0.6045 - val_accuracy: 0.7342
Epoch 3/10
19/19 [==============================] - 0s 4ms/step - loss: 0.9535 - accuracy: 0.6282 - val_loss: 0.5666 - val_accuracy: 0.7342
Epoch 4/10
19/19 [==============================] - 0s 4ms/step - loss: 0.7886 - accuracy: 0.6848 - val_loss: 0.5302 - val_accuracy: 0.7152
Epoch 5/10
19/19 [==============================] - 0s 3ms/step - loss: 0.7556 - accuracy: 0.6856 - val_loss: 0.5313 - val_accuracy: 0.7342
Epoch 6/10
19/19 [==============================] - 0s 4ms/step - loss: 0.7555 - accuracy: 0.6981 - val_loss: 0.6351 - val_accuracy: 0.7342
Epoch 7/10
19/19 [==============================] - 0s 3ms/step - loss: 0.6037 - accuracy: 0.7187 - val_loss: 0.5155 - val_accuracy: 0.7278
Epoch 8/10
19/19 [============

In [37]:
prediction_raw = model.predict(test_ds)

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


In [38]:
prediction_list = prediction_raw.squeeze().tolist()

In [39]:
test_df['predicted'] = prediction_list

In [40]:
# Put predicted as first col, next to survived.
cols = list(test_df.columns)
cols = [cols[-1]] + cols[:-1]
test_df = test_df[cols]

In [41]:
test_df

,predicted,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
96,0.344155,1,male,3.0,1,1,15.9000,Third,unknown,Southampton,n
200,0.137627,0,male,25.0,0,0,7.2250,Third,unknown,Cherbourg,y
175,0.543206,0,female,26.0,1,0,16.1000,Third,unknown,Southampton,n
97,0.054931,0,male,42.0,0,0,8.6625,Third,unknown,Southampton,y
213,0.439191,1,female,28.0,0,0,7.7375,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...,...
69,0.597751,1,female,29.0,0,2,15.2458,Third,unknown,Cherbourg,n
154,0.447966,1,male,8.0,1,1,36.7500,Second,unknown,Southampton,n
136,0.245601,0,male,55.0,0,0,30.5000,First,C,Southampton,y
211,0.186822,0,male,34.0,0,0,13.0000,Second,unknown,Southampton,y


In [42]:
OUTPUT_PATH = './titanic-fairness'

In [43]:
# Specify Fairness Indicators in eval_config.
eval_config = text_format.Parse("""
  model_specs {
    prediction_key: 'predicted',
    label_key: 'survived'
  }
  metrics_specs {
    metrics {class_name: "AUC"}
    metrics {
      class_name: "FairnessIndicators"
      config: '{"thresholds": [0.1, 0.50, 0.90]}'
    }
    metrics { class_name: "ExampleCount" }
  }
  
  slicing_specs {
    feature_keys: ['sex', 'class']
  }
  slicing_specs {}
  """, tfma.EvalConfig())

# Run TensorFlow Model Analysis.
eval_result = tfma.analyze_raw_data(
  data= test_df,
  eval_config=eval_config,
  output_path=OUTPUT_PATH)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [45]:
!jupyter nbextension enable tensorflow_model_analysis --user --py

2021-04-29 20:24:07.221567: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-04-29 20:24:07.221609: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Enabling notebook extension tensorflow_model_analysis/extension...
      - Validating: OK


In [46]:
# Render Fairness Indicators.
tfma.addons.fairness.view.widget_view.render_fairness_indicator(eval_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Third_X_male', 'slice': 'class_X_sex:Third_X_male', 'm…